In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import re
import requests
pd.options.display.max_rows = 5000 

- spaq : Spinaker Aquisitions

In [2]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/spaq.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

spaq = requests.request("GET", url, headers=headers)

# print(response.text[:500])

- pre : Pensana

In [3]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/pre.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

pre = requests.request("GET", url, headers=headers)

In [4]:
spaq = spaq.text
pre = pre.text

In [5]:
def get_dates(string):
    date = re.compile('"date":((.*),)')
    date = date.findall(string)
    date = [i[0][2:12] for i in date]
    date = [datetime.strptime(i, "%d-%m-%Y").strftime("%Y-%m-%d") for i in date]
    return date

def get_close(string):
    close = re.compile('"close":((.*),)')
    close = close.findall(string)
    close = [float(i[1].strip()) for i in close]
    return close

In [6]:
spaq_date = get_dates(spaq)
spaq_close = get_close(spaq)

In [7]:
pre_date = get_dates(pre)
pre_close = get_close(pre)

In [8]:
print(spaq_date[-1])
print(spaq_close[-1])
print(pre_date[-1])
print(pre_close[-1])

2021-08-12
15.5
2021-08-12
84.8


In [9]:
fund = 10000
client = ['Ma', 'Ma']
stock = ['spaq', 'pre']
total_cap = [10000, 10000]
invest_in = [5000, 5000]
ipo_price = [0.1, 1.34]
w_pirce = [0.2, 0]
s_position = pd.Series(invest_in)/pd.Series(ipo_price)
p_vs_w = [0.5, 0]
w_position = pd.Series(s_position)*pd.Series(p_vs_w)
s_date = [spaq_date[-1], pre_date[-1]]
s_close = [spaq_close[-1]/100, pre_close[-1]/100]

In [10]:
mr_ma = pd.DataFrame({'client':client, 'stock':stock, 'total_cap':total_cap, 'invest_in':invest_in, \
             'ipo_price':ipo_price, 'w_pirce':w_pirce ,'p_vs_w':p_vs_w,'s_position':s_position, \
             'w_position':w_position, 's_date':s_date, 's_close':s_close,\
             })

In [11]:
mr_ma['s_pft'] = (mr_ma['s_close'] - mr_ma['ipo_price'])* mr_ma['s_position']
mr_ma['w_pft'] = [(i-j)*k if i>j else 0 for i,j,k in zip(mr_ma.s_close, mr_ma.w_pirce, mr_ma.w_position)]

In [12]:
mr_ma

,client,stock,total_cap,invest_in,ipo_price,w_pirce,p_vs_w,s_position,w_position,s_date,s_close,s_pft,w_pft
0,Ma,spaq,10000,5000,0.10,0.2,0.5,50000.000000,25000.0,2021-08-12,0.155,2750.000000,0.0
1,Ma,pre,10000,5000,1.34,0.0,0.0,3731.343284,0.0,2021-08-12,0.848,-1835.820896,0.0


In [13]:
account_balance = fund + sum(mr_ma['s_pft'])+sum(mr_ma['w_pft'])
print(account_balance)

10914.179104477611
